This notebook will serve to house a non-ML method of classifying subaerial and subaqeuous land pixels from Sentinel-1 imagery. The imagery used will be Mon-temporally speckled filtered using Refined Lee 5x5 kernel size filter. The VV, VH, two normalized ratios, and several GLCM features will be used.
1. Apply unimodal (or bimodal) Otsu to the histograms of the various bands
2. Develope a probability map for each epoch based on the otsu classifications derived from each band for each pixel
3. Threshold the probability histogram for each pixel to get final classification

In [19]:
import numpy as np
from skimage.filters import threshold_otsu
import os
from datetime import datetime
from itertools import product
import rasterio
from rasterio import windows
from shapely.geometry import box
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from osgeo import gdal

# Functions

In [24]:
def get_grd(grdpath):
    orig_ims = [os.path.join(grdpath, file) for file in os.listdir(grdpath) if file.endswith('.tif')]

    return orig_ims

def get_glcm(glcmpath):
    orig_glcms = [os.path.join(glcmpath, file) for file in os.listdir(glcmpath) if file.endswith('.tif')]

    return orig_glcms

def load_tif_image(path):
    """Load a single-band image from a .tif file."""
    with rasterio.open(path) as src:
        return src.read(1)

def calculate_binary_mask(band):
    """
    Calculate the binary mask using Otsu's threshold for a given band.
    """
    otsu_thresh = threshold_otsu(band)
    binary_mask = (band >= otsu_thresh).astype(int)
    return binary_mask

def generate_probability_map(bands):
    """
    Generate the probability map from backscatter and GLCM features.
    
    Parameters:
    - vv, vh, ratio1, ratio2: 2D numpy arrays for backscatter bands
    - glcm_features: List of 2D numpy arrays for GLCM features
    
    Returns:
    - probability_map: 2D numpy array of probabilities
    """
    binary_masks = [calculate_binary_mask(band) for band in bands]
    
    # Calculate the probability map by averaging the binary masks
    probability_map = np.mean(binary_masks, axis=0)
    return probability_map


def final_classification(probability_map, adaptive=False):
    """
    Convert the probability map to a final binary classification.
    
    Parameters:
    - probability_map: 2D numpy array of probabilities
    - adaptive: Use adaptive thresholding (Otsu) if True, fixed threshold (0.5) if False

    Returns:
    - final_class: 2D numpy array of binary classification (1 = subaerial, 0 = subaqueous)
    """
    if adaptive:
        threshold = threshold_otsu(probability_map)
    else:
        threshold = 0.5

    # Apply the threshold to the probability map
    final_class = (probability_map >= threshold).astype(int)
    return final_class

def save_multiband_image_as_tiff(output_path, transformed_bands, reference_dataset, gdal_dtype=gdal.GDT_Float32):
    # Create an output GeoTIFF file with the same dimensions and the same number of bands
    driver = gdal.GetDriverByName('GTiff')
    out_dataset = driver.Create(output_path, reference_dataset.RasterXSize, reference_dataset.RasterYSize, len(transformed_bands), gdal_dtype)

    # Set the projection and geotransform from the reference dataset
    out_dataset.SetProjection(reference_dataset.GetProjection())
    out_dataset.SetGeoTransform(reference_dataset.GetGeoTransform())

    # Write each transformed band to the output file
    for i, transformed_band in enumerate(transformed_bands):
        out_dataset.GetRasterBand(i + 1).WriteArray(transformed_band)

    # Flush data to disk
    out_dataset.FlushCache()
    out_dataset = None

In [16]:
backscatter_ims = get_grd('/mnt/d/SabineRS/GRD/3_ratio')
glcm_ims = get_glcm('/mnt/d/SabineRS/GRD/2_registered/glcm')

In [34]:
probabilitymaps = []
classificationmaps = []

for i, im in enumerate(backscatter_ims):
    with rasterio.open(im) as grd_src:
        vv = grd_src.read(1)
        vh = grd_src.read(2)
        ratio1 = grd_src.read(3)
        ratio2 = grd_src.read(4)

    with rasterio.open(glcm_ims[i]) as glcm_src:
        # Extract GLCM features
        VV_contrast = glcm_src.read(1)  
        VV_asm = glcm_src.read(2) 
        VV_diss = glcm_src.read(3)  
        VV_idm = glcm_src.read(4)  
        VV_corr = glcm_src.read(5)  
        VV_var = glcm_src.read(6)  
        VV_ent = glcm_src.read(7) 
        VH_contrast = glcm_src.read(8)  
        VH_asm = glcm_src.read(9) 
        VH_diss = glcm_src.read(10)  
        VH_idm = glcm_src.read(11)  
        VH_corr = glcm_src.read(12)  
        VH_var = glcm_src.read(13)  
        VH_ent = glcm_src.read(14)
        glcm_features = [glcm_src.read(j) for j in range(1, 15)]

    # Generate the probability map for the current epoch
    # probability_map = generate_probability_map([vv, vh]+glcm_features)#+ glcm_features
    probability_map = generate_probability_map([vh, VH_contrast, VH_ent, VH_asm, VH_var])#+ glcm_features

    probabilitymaps.append(probability_map)
    final_class = final_classification(probability_map, adaptive=True)
    classificationmaps.append(final_class)

In [35]:
# Example usage to save probability and classification maps together
for i, (prob_map, class_map) in enumerate(zip(probabilitymaps, classificationmaps)):
    output_path = f'/mnt/d/SabineRS/s1classifications/combined_map_epoch_{i+1}.tif'
    reference_dataset = gdal.Open(backscatter_ims[i])

    # Combine probability map and final classification map into a list
    combined_bands = [prob_map, class_map]

    # Save the combined raster
    save_multiband_image_as_tiff(output_path, combined_bands, reference_dataset)

    print(f"Exported combined raster for epoch {i+1}")


Exported combined raster for epoch 1
Exported combined raster for epoch 2
Exported combined raster for epoch 3
Exported combined raster for epoch 4
Exported combined raster for epoch 5
Exported combined raster for epoch 6
Exported combined raster for epoch 7
Exported combined raster for epoch 8
Exported combined raster for epoch 9
Exported combined raster for epoch 10
Exported combined raster for epoch 11
Exported combined raster for epoch 12
Exported combined raster for epoch 13
Exported combined raster for epoch 14
Exported combined raster for epoch 15
Exported combined raster for epoch 16
Exported combined raster for epoch 17
Exported combined raster for epoch 18
Exported combined raster for epoch 19
Exported combined raster for epoch 20
Exported combined raster for epoch 21
Exported combined raster for epoch 22
Exported combined raster for epoch 23
Exported combined raster for epoch 24
Exported combined raster for epoch 25
Exported combined raster for epoch 26
Exported combined ras

In [ ]:
from scipy.ndimage import binary_opening, binary_closing, binary_fill_holes, generic_filter

def clean_classification(binary_class, structure_size=3):
    """
    Clean the binary classification using morphological operations and majority filter.
    
    Parameters:
    - binary_class: 2D numpy array of binary classification
    - structure_size: Size of the structuring element for morphological filtering

    Returns:
    - cleaned_class: 2D numpy array of cleaned classification
    """
    # Apply binary opening to remove small noise
    opened = binary_opening(binary_class, structure=np.ones((structure_size, structure_size)))

    # Apply binary closing to fill gaps
    closed = binary_closing(opened, structure=np.ones((structure_size, structure_size)))

    # Fill small holes
    filled = binary_fill_holes(closed)

    return filled

def majority_filter(binary_class, window_size=3):
    """
    Apply a majority filter to smooth isolated noise.
    """
    def majority_rule(window):
        return 1 if np.sum(window) > (window.size / 2) else 0

    return generic_filter(binary_class, majority_rule, size=window_size)

# Apply cleaning operations
cleaned_class = clean_classification(final_class, structure_size=3)
cleaned_class = majority_filter(cleaned_class, window_size=5)